In [15]:
from preprocessing import *
import preprocessing as prp
import pandas as pd

In [4]:
dataframe = pd.read_csv(r"C:\Users\carla\OneDrive\Documenti\GitHub\Unsupervised_Anomaly_Detection_thesis\Data\train_features.csv")
dataframe.shape

(1749494, 57)

In [5]:
df=dataframe[['building_id','primary_use', 'timestamp', 'meter_reading', 'sea_level_pressure', 'is_holiday','anomaly']]
df

,building_id,primary_use,timestamp,meter_reading,sea_level_pressure,is_holiday,anomaly
0,1,Education,2016-01-01 00:00:00,NaN,967.2,1,0
1,32,Office,2016-01-01 00:00:00,NaN,967.2,1,0
2,41,Office,2016-01-01 00:00:00,NaN,967.2,1,0
3,55,Office,2016-01-01 00:00:00,NaN,967.2,1,0
4,69,Parking,2016-01-01 00:00:00,NaN,967.2,1,0
...,...,...,...,...,...,...,...
1749489,1316,Entertainment/public assembly,2016-12-31 23:00:00,38.844,967.2,0,0
1749490,1318,Entertainment/public assembly,2016-12-31 23:00:00,202.893,967.2,0,0
1749491,1319,Entertainment/public assembly,2016-12-31 23:00:00,NaN,967.2,0,0
1749492,1323,Office,2016-12-31 23:00:00,172.000,967.2,0,0


In [18]:
from preprocessing import impute_nulls

ImportError: cannot import name 'impute_nulls' from 'preprocessing' (c:\Users\carla\OneDrive\Documenti\GitHub\Unsupervised_Anomaly_Detection_thesis\preprocessing.py)

In [19]:
imputed_df = impute_nulls(df)
imputed_df

NameError: name 'impute_nulls' is not defined